In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set()

In [ ]:
r = requests.post('http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/BefUtvKon1749')
r

In [ ]:
query_data = {
  "query": [
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1+2"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "0000001H"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post('http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/BefUtvKon1749',
                 json=query_data)
r

In [ ]:
json_data = r.json()['data']
json_data

In [ ]:
print ('record layout :',json_data[0])


In [ ]:
born_df = pd.DataFrame.from_dict(json_data)
born_df

In [ ]:
born_df[['code','year']] = born_df['key'].to_list()
born_df

In [ ]:
born_df['born'] = born_df['values'].apply(lambda x : x[0]).astype(int)
born_df

In [ ]:
born_df.drop(['key','values','code'],axis=1,inplace=True)

In [ ]:
born_df.set_index('year',inplace=True)
born_df

In [ ]:
born_df.loc['1900':'1950'].plot()

In [ ]:
dead_url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/BefUtvKon1749'
dead_query = {
  "query": [
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1+2"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "0000001F"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r_dead = requests.post(dead_url,json=dead_query)
r_dead

In [ ]:
json_data_dead = r_dead.json()['data']
json_data_dead

In [ ]:
dead_df = pd.DataFrame.from_dict(json_data_dead)
dead_df

In [ ]:
dead_df[['tmp','year']] = dead_df['key'].to_list()
dead_df

In [ ]:
dead_df['dead'] = dead_df['values'].apply(lambda x: x[0]).astype(int)
dead_df.drop(['key','values','tmp'],axis=1,inplace=True)
dead_df

In [ ]:
dead_df.set_index('year',inplace=True)
dead_df

In [ ]:
title = 'SWEDEN DEMOGRAPHICS\nyearly births & deaths 1749 - 2020\nDataSource : SCB.se'
ax = born_df.plot(figsize=(18,12),style='o--',title=title)
dead_df.plot(ax=ax,style='o--')
plt.ylabel('nr of events')
plt.savefig('scb_sweden_historical_births.jpg',format='jpg')

In [ ]:

excess_births = born_df['born'] - dead_df['dead']

ax = excess_births.plot(figsize=(18,12),color='g',ls='dashed',marker='o' )
ax.axhline(0,ls='dashed',color='k')

In [ ]:
born_df.loc['1900':].sort_values('born').head(40)

In [ ]:
### population ###

pop_url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860N'
pop_query = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:ÅlderTotA",
        "values": []
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(pop_url,json=pop_query)
r

In [ ]:
json_data = r.json()['data']
json_data[0]

In [ ]:
pop_df = pd.DataFrame.from_dict(json_data)
pop_df[['gender','year']] = pop_df['key'].to_list()
pop_df['pop'] = pop_df['values'].apply(lambda x : x[0]).astype(int)
pop_df.drop(['key','values'],axis=1,inplace=True)
pop_df

In [ ]:
pop_df = pop_df.groupby('year').sum()
pop_df

In [ ]:
pop_df.shift()

In [ ]:

births_per_pop = (born_df['born'] / pop_df['pop'].shift()).dropna()
births_per_pop

In [ ]:
title = 'SWEDEN DEMOGRAPHICS\nproportion yearly births per total population 1861 - 2020\nDataSource : SCB.se'

births_per_pop.plot(figsize=(18,12),style='o--',title=title)
plt.ylabel("births/pop")
plt.savefig('scb_sweden_historical_births_prop.jpg',format='jpg')

In [ ]:
births_per_pop.sort_values().head(40)

In [ ]:
pop_increase_by_births = (excess_births / pop_df['pop'].shift()).dropna()
pop_increase_by_births

In [ ]:
ax = pop_increase_by_births.plot(figsize=(18,12),ls='dashed',marker='o',color='orange')
ax.axhline(0,color='k',ls='dashed')